In [18]:
import pyspark

In [19]:
from pyspark.sql import SparkSession

In [20]:
spark = SparkSession.builder.appName("SocialNetworkAnalysis").config("spark.jars", "C:/Big Data/graphframes/graphframes-0.8.2-spark3.0-s_2.12.jar").getOrCreate()

In [21]:
spark

In [ ]:
spark.sparkContext.setCheckpointDir(r"C:\Master-Projects\Social_Media_BigData\checkpoints")

In [ ]:
df = spark.read.text("facebook_combined.txt.gz")

In [ ]:
df.show()

In [ ]:
from pyspark.sql.functions import split, col
from graphframes import GraphFrame

In [ ]:
edges = df.withColumn("src", split(col("value"), " ")[0]).withColumn("dst", split(col("value"), " ")[1]).select("src", "dst")

In [ ]:
vertices = edges.select(col("src").alias("id")).union(edges.select(col("dst").alias("id"))).distinct()

In [ ]:
graph = GraphFrame(vertices, edges)

* a) Si mund të identifikojmë përdoruesit më të ndikshëm duke përdorur algoritme të centralitetit?

In [ ]:
# Apliko PageRank
pagerank = graph.pageRank(resetProbability=0.15, maxIter=10)
pagerank.vertices.orderBy(col("pagerank").desc()).show(10, truncate=False)

b) Si mund të përdoren grafët për të zbuluar grupe të ngjashme përdoruesish në një rrjet social?

In [ ]:
graph.edges.show()

In [ ]:
graph.vertices.show()

In [ ]:
connected_components = graph.connectedComponents()

In [ ]:
# Gjej komponentët e lidhur
spark.sparkContext.setCheckpointDir(r"C:\Big Data\checkpoint")
graph = GraphFrame(vertices, edges)
connected_components = graph.connectedComponents()
connected_components.orderBy("component").show(10, truncate=False)

c) Si mund të përdorim algoritmet e përhapjes së informacionit për të analizuar shpërndarjen e lajmeve ose viruseve?

In [ ]:
# Apliko algoritmin e përhapjes së informacionit
label_propagation = graph.labelPropagation(maxIter=5)
label_propagation.orderBy("label").show(10, truncate=False)

d) Çfarë ndikimi ka heqja e një përdoruesi kyç në strukturën dhe stabilitetin e rrjetit social?

In [ ]:
# Hiq një përdorues nga grafi
user_to_remove = "0"  # P.sh., përdoruesi me ID 0
updated_edges = edges.filter((col("src") != user_to_remove) & (col("dst") != user_to_remove))
updated_vertices = vertices.filter(col("id") != user_to_remove)
updated_graph = GraphFrame(updated_vertices, updated_edges)

# Analizo strukturën e re
updated_graph.connectedComponents().show()

e) Si mund të përdorim PageRank për të vlerësuar ndikimin e përdoruesve në një rrjet social?